# Regularisasi

<a name='1'></a>
## Packages

In [ ]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import scipy.io
from reg_utils import sigmoid, relu, plot_decision_boundary, initialize_parameters, load_2D_dataset, predict_dec
from reg_utils import compute_cost, predict, forward_propagation, backward_propagation, update_parameters
from testCases import *
from public_tests import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

<a name='2'></a>
## Masalah

Anda baru saja dipekerjakan sebagai ahli AI oleh French Football Corporation. Mereka ingin Anda merekomendasikan posisi di mana penjaga gawang Prancis harus menendang bola sehingga pemain tim Prancis dapat menerimanya dengan kepala.

<img src="images/field_kiank.png" style="width:600px;height:350px;">

<caption><center><font color='purple'><b>Gambar 1</b>: Lapangan sepak bola. Penjaga gawang menendang bola ke udara, pemain masing-masing tim berebut bola dengan kepala </font></center></caption>


Mereka memberi Anda kumpulan data 2D berikut dari 10 pertandingan terakhir Prancis.

<a name='3'></a>
## Dataset

In [ ]:
train_X, train_Y, test_X, test_Y = load_2D_dataset()

Setiap titik sesuai dengan posisi di lapangan sepak bola di mana seorang pemain sepak bola menerima bola dengan kepalanya setelah penjaga gawang Prancis menembak bola dari sisi kiri lapangan sepak bola.
- Jika titik berwarna biru berarti pemain Perancis tersebut berhasil mengenai bola dengan kepalanya
- Jika titik berwarna merah berarti pemain tim lain mengenai bola dengan kepalanya

**Tujuan Anda**: Gunakan model deep learning untuk menemukan posisi di lapangan di mana kiper harus menendang bola.

**Analisis kumpulan data**: Kumpulan data ini sedikit kotor, namun sepertinya garis diagonal yang memisahkan separuh kiri atas dari separuh kanan bawah akan berfungsi dengan baik.

Pertama-tama Anda akan mencoba model tanpa regularisasi. Kemudian Anda akan belajar cara menggunakan regularisasi dan memutuskan model mana yang akan Anda pilih untuk menyelesaikan masalah Perusahaan Sepak Bola Prancis.

<a name='4'></a>
## Model Tanpa Regularisasi

Anda akan menggunakan neural network berikut (sudah diimplementasikan untuk Anda di bawah). Model ini dapat digunakan:
- dalam *mode regularisasi* -- dengan mengatur input `lambd` ke nilai bukan nol. Kami menggunakan "`lambd`" dan bukan "`lambda`" karena "`lambda`" adalah keyword khusus dalam Python.
- dalam *mode dropout* -- dengan menyetel `keep_prob` ke nilai kurang dari satu

Anda pertama-tama akan mencoba model tersebut tanpa regularisasi apa pun. Kemudian, Anda akan menerapkan:
- *regularisasi L2* -- fungsi: "`compute_cost_with_regularization()`" dan "`backward_propagation_with_regularization()`"
- *Dropout* -- fungsi: "`forward_propagation_with_dropout()`" dan "`backward_propagation_with_dropout()`"

Di setiap bagian, Anda akan menjalankan model ini dengan input yang benar sehingga proses ini akan memanggil fungsi yang telah Anda implementasikan. Lihatlah kode di bawah ini untuk membiasakan diri Anda dengan modelnya.

In [ ]:
def model(X, Y, learning_rate = 0.3, num_iterations = 30000, print_cost = True, lambd = 0, keep_prob = 1):
    """
    Implements a three-layer neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (output size, number of examples)
    learning_rate -- learning rate of the optimization
    num_iterations -- number of iterations of the optimization loop
    print_cost -- If True, print the cost every 10000 iterations
    lambd -- regularization hyperparameter, scalar
    keep_prob - probability of keeping a neuron active during drop-out, scalar.
    
    Returns:
    parameters -- parameters learned by the model. They can then be used to predict.
    """
        
    grads = {}
    costs = []                            # to keep track of the cost
    m = X.shape[1]                        # number of examples
    layers_dims = [X.shape[0], 20, 3, 1]
    
    # Initialize parameters dictionary.
    parameters = initialize_parameters(layers_dims)

    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # Forward propagation: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID.
        if keep_prob == 1:
            a3, cache = forward_propagation(X, parameters)
        elif keep_prob < 1:
            a3, cache = forward_propagation_with_dropout(X, parameters, keep_prob)
        
        # Cost function
        if lambd == 0:
            cost = compute_cost(a3, Y)
        else:
            cost = compute_cost_with_regularization(a3, Y, parameters, lambd)
            
        # Backward propagation.
        assert (lambd == 0 or keep_prob == 1)   # it is possible to use both L2 regularization and dropout, 
                                                # but this assignment will only explore one at a time
        if lambd == 0 and keep_prob == 1:
            grads = backward_propagation(X, Y, cache)
        elif lambd != 0:
            grads = backward_propagation_with_regularization(X, Y, cache, lambd)
        elif keep_prob < 1:
            grads = backward_propagation_with_dropout(X, Y, cache, keep_prob)
        
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Print the loss every 10000 iterations
        if print_cost and i % 10000 == 0:
            print("Cost after iteration {}: {}".format(i, cost))
        if print_cost and i % 1000 == 0:
            costs.append(cost)
    
    # plot the cost
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations (x1,000)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

Mari kita latih modelnya tanpa regularisasi apa pun, dan amati keakuratannya pada set latih/uji.

In [ ]:
parameters = model(train_X, train_Y)
print ("On the training set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

Akurasi latih 94,8% sedangkan akurasi uji 91,5%. Ini adalah **model dasar** (Anda akan mengamati dampak regularisasi pada model ini). Jalankan kode berikut untuk memplot batas keputusan model Anda.

In [ ]:
plt.title("Model without regularization")
axes = plt.gca()
axes.set_xlim([-0.75,0.40])
axes.set_ylim([-0.75,0.65])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

Model tanpa regularisasi jelas-jelas melampaui set latih. Ini karena proses fitting dengan noise! Sekarang mari kita lihat dua teknik untuk mengurangi overfitting.

<a name='5'></a>
## Regularisasi L2

Cara standar untuk menghindari overfitting disebut **regularisasi L2**. Ini terdiri dari memodifikasi fungsi cost Anda dengan tepat, dari:
$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small  y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} \tag{1}$$
menuju:
$$J_{regularized} = \small \underbrace{-\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} }_\text{cross-entropy cost} + \underbrace{\frac{1}{m} \frac{\lambda}{2} \sum\limits_l\sum\limits_k\sum\limits_j W_{k,j}^{[l]2} }_\text{L2 regularization cost} \tag{2}$$

Mari ubah cost dan amati konsekuensinya.

<a name='ex-1'></a>
### Tugas 1 - compute_cost_with_regularization
Terapkan `compute_cost_with_regularization()` yang menghitung cost yang diberikan oleh rumus (2). Untuk menghitung $\sum\limits_k\sum\limits_j W_{k,j}^{[l]2}$ , gunakan:
```
np.sum(np.square(Wl))
```
Perhatikan bahwa Anda harus melakukan ini untuk $W^{[1]}$, $W^{[2]}$ dan $W^{[3]}$, lalu jumlahkan ketiga suku tersebut dan kalikan dengan $ \frac{1}{m} \frac{\lambda}{2} $.

In [ ]:
# GRADED FUNCTION: compute_cost_with_regularization

def compute_cost_with_regularization(A3, Y, parameters, lambd):
    """
    Implement the cost function with L2 regularization. See formula (2) above.
    
    Arguments:
    A3 -- post-activation, output of forward propagation, of shape (output size, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    parameters -- python dictionary containing parameters of the model
    
    Returns:
    cost - value of the regularized loss function (formula (2))
    """
    m = Y.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]
    
    cross_entropy_cost = compute_cost(A3, Y) # This gives you the cross-entropy part of the cost
    
    #(≈ 1 lines of code)
    # L2_regularization_cost = 
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    
    cost = cross_entropy_cost + L2_regularization_cost
    
    return cost

In [ ]:
A3, t_Y, parameters = compute_cost_with_regularization_test_case()
cost = compute_cost_with_regularization(A3, t_Y, parameters, lambd=0.1)
print("cost = " + str(cost))

compute_cost_with_regularization_test(compute_cost_with_regularization)

Tentu saja, karena Anda mengubah cost, Anda juga harus mengubah propagasi backward! Semua gradien harus dihitung sehubungan dengan cost baru ini.

<a name='ex-2'></a>
### Tugas 2 - backward_propagation_with_regularization
Menerapkan perubahan yang diperlukan dalam propagasi backward untuk memperhitungkan regularisasi. Perubahan tersebut hanya menyangkut dW1, dW2 dan dW3. Untuk masing-masingnya, Anda harus menambahkan gradien dari suku regularisasi ($\frac{d}{dW} ( \frac{1}{2}\frac{\lambda}{m}  W^2) = \frac{\lambda}{m} W$).

In [ ]:
# GRADED FUNCTION: backward_propagation_with_regularization

def backward_propagation_with_regularization(X, Y, cache, lambd):
    """
    Implements the backward propagation of our baseline model to which we added an L2 regularization.
    
    Arguments:
    X -- input dataset, of shape (input size, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    cache -- cache output from forward_propagation()
    lambd -- regularization hyperparameter, scalar
    
    Returns:
    gradients -- A dictionary with the gradients with respect to each parameter, activation and pre-activation variables
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    #(≈ 1 lines of code)
    # dW3 = 1./m * np.dot(dZ3, A2.T) + None
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    #(≈ 1 lines of code)
    # dW2 = 1./m * np.dot(dZ2, A1.T) + None
    # YOUR CODE STARTS HERE
    
    # YOUR CODE ENDS HERE
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    #(≈ 1 lines of code)
    # dW1 = 1./m * np.dot(dZ1, X.T) + None
    # YOUR CODE STARTS HERE

    
    # YOUR CODE ENDS HERE
    db1 = 1. / m * np.sum(dZ1, axis=1, keepdims=True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
                 "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1, 
                 "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [ ]:
t_X, t_Y, cache = backward_propagation_with_regularization_test_case()

grads = backward_propagation_with_regularization(t_X, t_Y, cache, lambd = 0.7)
print ("dW1 = \n"+ str(grads["dW1"]))
print ("dW2 = \n"+ str(grads["dW2"]))
print ("dW3 = \n"+ str(grads["dW3"]))
backward_propagation_with_regularization_test(backward_propagation_with_regularization)

Sekarang mari kita jalankan model dengan regularisasi L2 $(\lambda = 0.7)$. Fungsi `model()` akan memanggil:
- `compute_cost_with_regularization` bukannya `compute_cost`
- `backward_propagation_with_regularization` bukannya `backward_propagation`

In [ ]:
parameters = model(train_X, train_Y, lambd = 0.7)
print ("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

Selamat, akurasi set uji meningkat menjadi 93%. Anda telah menyelamatkan tim sepak bola Prancis!

Anda tidak lagi melakukan overfitting pada data latih. Mari kita gambarkan batasan keputusannya.

In [ ]:
plt.title("Model with L2-regularization")
axes = plt.gca()
axes.set_xlim([-0.75,0.40])
axes.set_ylim([-0.75,0.65])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

**Pengamatan**:
- Nilai $\lambda$ adalah hyperparameter yang dapat Anda sesuaikan menggunakan set dev.
- Regularisasi L2 membuat batasan keputusan Anda lebih halus. Jika $\lambda$ terlalu besar, hal ini juga memungkinkan untuk "terlalu halus", sehingga menghasilkan model dengan bias yang tinggi.

**Apa yang sebenarnya dilakukan regularisasi L2?**:

Regularisasi L2 mengandalkan asumsi bahwa model dengan bobot kecil lebih sederhana dibandingkan model dengan bobot besar. Jadi, dengan memberi penalti pada nilai kuadrat bobot dalam fungsi cost, Anda mengarahkan semua bobot ke nilai yang lebih kecil. Menjadi terlalu mahal untuk memiliki bobot yang besar! Hal ini menghasilkan model yang lebih halus di mana output berubah lebih pelan seiring dengan perubahan input.

<br>
<font color='blue'>
    
**Yang harus Anda ingat:** implikasi regularisasi L2 pada:
- Perhitungan cost:
    - Istilah regularisasi ditambahkan ke cost.
- Fungsi backpropagation:
    - Ada suku tambahan dalam gradien sehubungan dengan matriks bobot.
- Bobot menjadi lebih kecil ("peluruhan bobot"):
    - Bobot didorong ke nilai yang lebih kecil.

<a name='6'></a>
## Dropout

Terakhir, **dropout** adalah teknik regularisasi yang banyak digunakan dan khusus untuk deep learning.
**Ini secara acak mematikan beberapa neuron di setiap iterasi.** Tonton dua video ini untuk mengetahui apa artinya!


<center>
<video width="620" height="440" src="images/dropout1_kiank.mp4" type="video/mp4" controls>
</video>
</center>
<br>
<caption><center><font color='purple'><b>Gambar 2 </b>: <b>Drop-out pada hidden layer kedua.</b> <br> Pada setiap iterasi, Anda mematikan (= menyetel ke nol) setiap neuron lapisan dengan probabilitas $1 - keep\_prob$ atau menyimpannya dengan probabilitas $keep\_prob$ (50% di sini). Neuron yang dijatuhkan tidak berkontribusi pada pelatihan baik dalam propagasi iterasi forward maupun backward. </font></center></caption>

<center>
<video width="620" height="440" src="images/dropout2_kiank.mp4" type="video/mp4" controls>
</video>
</center>

<caption><center><font color='purple'><b>Gambar 3</b>:<b> Drop-out pada hidden layers pertama dan ketiga. </b><br> $1^{st}$: kita mematikan rata-rata 40% neuron. lapisan $3^{rd}$: kita mematikan rata-rata 20% neuron. </font></center></caption>


Saat Anda mematikan beberapa neuron, Anda sebenarnya memodifikasi model Anda. Ide di balik drop-out adalah pada setiap iterasi, Anda melatih model berbeda yang hanya menggunakan sebagian neuron Anda. Dengan dropout, neuron Anda menjadi kurang sensitif terhadap aktivasi satu neuron spesifik lainnya, karena neuron lain tersebut dapat dimatikan kapan saja.

<a name='6-1'></a>
### Propagasi Forward dengan Dropout

<a name='ex-3'></a>
### Tugas 3 - forward_propagation_with_dropout

Menerapkan propagasi forward dengan dropout. Anda menggunakan neural network 3 lapisan, dan akan menambahkan dropout ke hidden layer pertama dan kedua. Dropout tidak digunakan pada lapisan input atau lapisan output.

**Petunjuk**:
Anda ingin mematikan beberapa neuron di lapisan pertama dan kedua. Untuk melakukan itu, Anda akan melakukan 4 Langkah:
1. Kita membahas pembuatan variabel $d^{[1]}$ dengan bentuk yang sama seperti $a^{[1]}$ menggunakan `np.random.rand()` untuk mendapatkan angka antara 0 dan 1 secara acak. Di sini, Anda akan menggunakan implementasi vektor, jadi buatlah matriks acak $D^{[1]} = [d^{[1](1)} d^{[1](2)} ... d^{[1](m)}] $ dengan dimensi yang sama dengan $A^{[1]}$.
2. Tetapkan setiap entri $D^{[1]}$ menjadi 1 dengan probabilitas (`keep_prob`), dan 0 sebaliknya.

**Petunjuk:** Katakanlah keep_prob = 0.8, yang berarti kita ingin mempertahankan sekitar 80% neuron dan membuang sekitar 20% neuron tersebut. Kita ingin membuat vektor yang mempunyai angka 1 dan 0, dimana sekitar 80% diantaranya adalah 1 dan sekitar 20% adalah 0.
Pernyataan python ini:
`X = (X < keep_prob).astype(int)`

secara konseptual sama dengan pernyataan if-else ini (untuk kasus sederhana array satu dimensi):

```
for i,v in enumerate(x):
    if v < keep_prob:
        x[i] = 1
    else: # v >= keep_prob
        x[i] = 0
```
Perhatikan bahwa `X = (X < keep_prob).astype(int)` berfungsi dengan array multidimensi, dan output yang dihasilkan mempertahankan dimensi array input.

Perhatikan juga bahwa tanpa menggunakan `.astype(int)`, hasilnya adalah array boolean `True` dan `False`, yang secara otomatis dikonversi oleh Python menjadi 1 dan 0 jika kita mengalikannya dengan angka. (Namun, praktik yang lebih baik adalah mengonversi data ke tipe data yang kita inginkan, jadi coba gunakan `.astype(int)`.)

3. Setel $A^{[1]}$ ke $A^{[1]} * D^{[1]}$. (Anda mematikan beberapa neuron). Anda dapat menganggap $D^{[1]}$ sebagai penutup, sehingga ketika dikalikan dengan matriks lain, beberapa nilainya akan mati.
4. Bagilah $A^{[1]}$ dengan `keep_prob`. Dengan melakukan hal ini, Anda yakin bahwa hasil cost akan tetap mempunyai nilai yang diharapkan sama seperti tanpa dropout. (Teknik ini juga disebut dropout terbalik.)

In [ ]:
# GRADED FUNCTION: forward_propagation_with_dropout

def forward_propagation_with_dropout(X, parameters, keep_prob = 0.5):
    """
    Implements the forward propagation: LINEAR -> RELU + DROPOUT -> LINEAR -> RELU + DROPOUT -> LINEAR -> SIGMOID.
    
    Arguments:
    X -- input dataset, of shape (2, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (20, 2)
                    b1 -- bias vector of shape (20, 1)
                    W2 -- weight matrix of shape (3, 20)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)
    keep_prob - probability of keeping a neuron active during drop-out, scalar
    
    Returns:
    A3 -- last activation value, output of the forward propagation, of shape (1,1)
    cache -- tuple, information stored for computing the backward propagation
    """
    
    np.random.seed(1)
    
    # retrieve parameters
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    #(≈ 4 lines of code)         # Steps 1-4 below correspond to the Steps 1-4 described above. 
    # D1 =                                           # Step 1: initialize matrix D1 = np.random.rand(..., ...)
    # D1 =                                           # Step 2: convert entries of D1 to 0 or 1 (using keep_prob as the threshold)
    # A1 =                                           # Step 3: shut down some neurons of A1
    # A1 =                                           # Step 4: scale the value of neurons that haven't been shut down
    # YOUR CODE STARTS HERE
    
    # YOUR CODE ENDS HERE
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    #(≈ 4 lines of code)
    # D2 =                                           # Step 1: initialize matrix D2 = np.random.rand(..., ...)
    # D2 =                                           # Step 2: convert entries of D2 to 0 or 1 (using keep_prob as the threshold)
    # A2 =                                           # Step 3: shut down some neurons of A2
    # A2 =                                           # Step 4: scale the value of neurons that haven't been shut down
    # YOUR CODE STARTS HERE
    
    # YOUR CODE ENDS HERE
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    cache = (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3)
    
    return A3, cache

In [ ]:
t_X, parameters = forward_propagation_with_dropout_test_case()

A3, cache = forward_propagation_with_dropout(t_X, parameters, keep_prob=0.7)
print ("A3 = " + str(A3))

forward_propagation_with_dropout_test(forward_propagation_with_dropout)

<a name='6-2'></a>
### Propagasi Backward dengan Dropout

<a name='ex-4'></a>
### Tugas 4 - backward_propagation_with_dropout

Menerapkan propagasi backward dengan dropout. Seperti sebelumnya, Anda sedang melatih network 3 lapis. Tambahkan dropout ke hidden layer pertama dan kedua, menggunakan penutup $D^{[1]}$ dan $D^{[2]}$ yang disimpan dalam cache.

**Petunjuk**:
Propagasi backward dengan dropout sebenarnya cukup mudah. Anda harus melakukan 2 Langkah:
1. Sebelumnya Anda telah mematikan beberapa neuron selama propagasi forward, dengan menerapkan penutup $D^{[1]}$ ke `A1`. Dalam propagasi mundur, Anda harus mematikan neuron yang sama, dengan menerapkan kembali penutup $D^{[1]}$ yang sama ke `dA1`.
2. Selama propagasi forward, Anda membagi `A1` dengan `keep_prob`. Oleh karena itu, dalam propagasi backward, Anda harus membagi `dA1` dengan `keep_prob` lagi (interpretasi kalkulusnya adalah jika $A^{[1]}$ diskalakan dengan `keep_prob`, maka turunannya $dA^{[1]}$ juga diskalakan oleh sama `keep_prob`).

In [ ]:
# GRADED FUNCTION: backward_propagation_with_dropout

def backward_propagation_with_dropout(X, Y, cache, keep_prob):
    """
    Implements the backward propagation of our baseline model to which we added dropout.
    
    Arguments:
    X -- input dataset, of shape (2, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    cache -- cache output from forward_propagation_with_dropout()
    keep_prob - probability of keeping a neuron active during drop-out, scalar
    
    Returns:
    gradients -- A dictionary with the gradients with respect to each parameter, activation and pre-activation variables
    """
    
    m = X.shape[1]
    (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims=True)
    dA2 = np.dot(W3.T, dZ3)
    #(≈ 2 lines of code)
    # dA2 =                # Step 1: Apply mask D2 to shut down the same neurons as during the forward propagation
    # dA2 =                # Step 2: Scale the value of neurons that haven't been shut down
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(W2.T, dZ2)
    #(≈ 2 lines of code)
    # dA1 =                # Step 1: Apply mask D1 to shut down the same neurons as during the forward propagation
    # dA1 =                # Step 2: Scale the value of neurons that haven't been shut down
    # YOUR CODE STARTS HERE
    
    # YOUR CODE ENDS HERE
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims=True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
                 "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1, 
                 "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [ ]:
t_X, t_Y, cache = backward_propagation_with_dropout_test_case()

gradients = backward_propagation_with_dropout(t_X, t_Y, cache, keep_prob=0.8)

print ("dA1 = \n" + str(gradients["dA1"]))
print ("dA2 = \n" + str(gradients["dA2"]))

backward_propagation_with_dropout_test(backward_propagation_with_dropout)

Sekarang mari kita jalankan model dengan dropout (`keep_prob = 0.86`). Artinya pada setiap iterasi, Anda mematikan masing-masing neuron pada lapisan 1 dan 2 dengan probabilitas 14%. Fungsi `model()` sekarang akan memanggil:
- `forward_propagation_with_dropout` bukannya `forward_propagation`.
- `backward_propagation_with_dropout` bukannya `backward_propagation`.

In [ ]:
parameters = model(train_X, train_Y, keep_prob = 0.86, learning_rate = 0.3)

print ("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

Dropout bekerja dengan baik! Akurasi uji meningkat lagi (hingga 95%)! Model Anda tidak melakukan overfitting pada set pelatihan dan berfungsi dengan baik pada set pengujian. Tim sepak bola Prancis akan selamanya berterima kasih kepada Anda!

Jalankan kode di bawah ini untuk plot batas keputusan.

In [ ]:
plt.title("Model with dropout")
axes = plt.gca()
axes.set_xlim([-0.75,0.40])
axes.set_ylim([-0.75,0.65])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

**Catatan**:
- **kesalahan umum** saat menggunakan dropout adalah menggunakannya dalam pelatihan dan pengujian. Anda harus menggunakan dropout (menghilangkan node secara acak) hanya dalam pelatihan.
- Framework deep learning seperti [tensorflow](https://www.tensorflow.org/api_docs/python/tf/nn/dropout), [PaddlePaddle](http://doc.paddlepaddle.org/release<_>doc/0.9.0/doc/ui/api/trainer_config_helpers/attrs.html), [keras](https://keras.io/layers/core/#dropout) atau [ caffe](http://caffe.berkeleyvision.org/tutorial/layers/dropout.html) hadir dengan implementasi lapisan dropout. 

<font color='blue'>
    
<**>Yang harus Anda ingat tentang dropout:<**>
- Dropout adalah teknik regularisasi.
- Anda hanya menggunakan dropout selama pelatihan. Jangan gunakan dropout (hilangkan node secara acak) selama waktu pengujian.
- Terapkan dropout saat propagasi forward dan backward.
- Selama waktu pelatihan, bagi setiap lapisan dropout dengan keep_prob untuk menjaga nilai yang diharapkan sama untuk aktivasi. Misalnya, jika keep_prob adalah 0.5, maka kita akan menutup rata-rata separuh node, sehingga output akan diskalakan sebesar 0.5 karena hanya separuh sisanya yang berkontribusi terhadap solusi. Membagi dengan 0.5 sama dengan mengalikan dengan 2. Oleh karena itu, outputnya sekarang memiliki nilai yang diharapkan sama. Anda dapat memeriksa apakah ini berfungsi bahkan ketika keep_prob bernilai selain 0.5.

<a name='7'></a>
## Kesimpulan

**Berikut adalah hasil dari ketiga model**:

<table>
    <tr>
        <td>
        <b>model</b>
        </td>
        <td>
        <b>train accuracy</b>
        </td>
        <td>
        <b>test accuracy</b>
        </td>
    </tr>
        <td>
        NN 3 lapis tanpa regularisasi
        </td>
        <td>
        95%
        </td>
        <td>
        91,5%
        </td>
    <tr>
        <td>
        NN 3 lapis dengan regularisasi L2
        </td>
        <td>
        94%
        </td>
        <td>
        93%
        </td>
    </tr>
    <tr>
        <td>
        NN 3 lapis dengan dropout
        </td>
        <td>
        93%
        </td>
        <td>
        95%
        </td>
    </tr>
</table>

Perhatikan bahwa regularisasi merugikan kinerja set latih! Hal ini karena membatasi kemampuan network untuk melakukan overfit ke set latih. Namun karena pada akhirnya memberikan akurasi uji yang lebih baik, ini membantu sistem Anda.